In [1]:
 import pandas as pd
from openpyxl import load_workbook

# 加载数据
file_path = "E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx"
sheet_name = "Parent1-3"
df = pd.read_excel(file_path, sheet_name=sheet_name)
# 计算第一行从第六列开始，满足'A', 'B', 'H', '-'条件的单元格数量
total_cells = df.iloc[0, 5:].apply(lambda x: x in ['A', 'B', 'H', '-']).sum()



In [2]:
final_df=df
def find_bounds_for_all_rows(df):
    bounds = {}
    for index, row in df.iterrows():
        pos = row['POS-2']
        # 上界：当前位置减去102400
        upper_bound = df[df['POS-2'] <= pos - 102400].index.max() if not df[df['POS-2'] <= pos - 102400].empty else None
        # 下界：当前位置加上102400
        lower_bound = df[df['POS-2'] >= pos + 102400].index.min() if not df[df['POS-2'] >= pos + 102400].empty else None
        bounds[index] = (upper_bound, lower_bound)
    return bounds

def validate_rows_with_bounds(df, bounds):
    error_counts = []
    for index, row in df.iterrows():
        error_count = 0
        upper_bound_index, lower_bound_index = bounds.get(index, (None, None))
        if upper_bound_index is None or lower_bound_index is None:
            error_counts.append(error_count)
            continue
        for col in range(5, df.shape[1] - 4):  # 忽略最后4列的特殊列
            cell = row.iloc[col]
            if cell == '-':
                continue
            upper_cell = df.iloc[upper_bound_index, col] if upper_bound_index else None
            lower_cell = df.iloc[lower_bound_index, col] if lower_bound_index else None
            if upper_cell == '-' or lower_cell == '-':
                continue
            if cell != upper_cell and cell != lower_cell:
                error_count += 1
        error_counts.append(error_count)
    return error_counts

# 预计算每行的上下界索引
bounds = find_bounds_for_all_rows(final_df)

# 使用预计算的索引进行错误计数
error_counts = validate_rows_with_bounds(final_df, bounds)

# 将错误计数添加到DataFrame
final_df['Error_Count'] = error_counts


In [3]:
def find_bounds_for_all_rows_avoiding_errors(df, bounds, error_threshold=10):
    new_bounds = {}
    for index in bounds:
        # 获取原始的上下界索引
        upper_bound_index, lower_bound_index = bounds[index]

        # 避开Error_Count>10的行，查找新的上界
        while upper_bound_index is not None and df.at[upper_bound_index, 'Error_Count'] > error_threshold:
            upper_bound_index -= 1  # 往上移动
            if upper_bound_index < 0:
                upper_bound_index = None
                break

        # 避开Error_Count>10的行，查找新的下界
        while lower_bound_index is not None and df.at[lower_bound_index, 'Error_Count'] > error_threshold:
            lower_bound_index += 1  # 往下移动
            if lower_bound_index >= len(df):
                lower_bound_index = None
                break

        new_bounds[index] = (upper_bound_index, lower_bound_index)
    return new_bounds

def reevaluate_error_counts(df, bounds):
    error_counts = []
    for index, row in df.iterrows():
        upper_bound_index, lower_bound_index = bounds.get(index, (None, None))
        error_count = 0
        if upper_bound_index is None or lower_bound_index is None:
            error_counts.append(error_count)
            continue
        for col in range(5, df.shape[1] - 4):
            cell = row.iloc[col]
            if cell == '-' or df.iloc[upper_bound_index, col] == '-' or df.iloc[lower_bound_index, col] == '-':
                continue
            if cell != df.iloc[upper_bound_index, col] and cell != df.iloc[lower_bound_index, col]:
                error_count += 1
        error_counts.append(error_count)
    return error_counts

# 找出所有Error_Count大于10的行索引
high_error_indices = final_df[final_df['Error_Count'] > 10].index

# 预计算每行的上下界索引，避开Error_Count>10的行
bounds = find_bounds_for_all_rows(final_df)
new_bounds = find_bounds_for_all_rows_avoiding_errors(final_df, bounds)

# 使用新的上下界索引进行错误计数
new_error_counts = reevaluate_error_counts(final_df, new_bounds)

# 更新DataFrame的Error_Count列
for index, error_count in zip(final_df.index, new_error_counts):
    final_df.at[index, 'Error_Count'] = error_count

# 处理逻辑完成后，您可以根据需要继续处理df，比如删除Error_Count依然大于10的行


In [4]:
# 保存处理后的数据和失真行
with pd.ExcelWriter("E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx", engine='openpyxl', mode='a') as writer:
    final_df.to_excel(writer, sheet_name='Parent1-5', index=False)

In [5]:
 import pandas as pd
from openpyxl import load_workbook

# 加载数据
file_path = "E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx"
sheet_name = "Parent1-5"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 找到所有Error_Count列中数量大于等于10的行
rows_to_replace = df[df['Error_Count'] > 10].index

# 执行替换操作
for index in rows_to_replace:
    for col in range(5, df.shape[1]):  # 从第六列开始
        cell_value = df.iloc[index, col]
        # 直接将A替换成H，H替换成A
        if cell_value == 'A':
            df.iloc[index, col] = 'H'
        elif cell_value == 'H':
            df.iloc[index, col] = 'A'
final_df=df
def find_bounds_for_all_rows(df):
    bounds = {}
    for index, row in df.iterrows():
        pos = row['POS-2']
        # 上界：当前位置减去102400
        upper_bound = df[df['POS-2'] <= pos - 102400].index.max() if not df[df['POS-2'] <= pos - 102400].empty else None
        # 下界：当前位置加上102400
        lower_bound = df[df['POS-2'] >= pos + 102400].index.min() if not df[df['POS-2'] >= pos + 102400].empty else None
        bounds[index] = (upper_bound, lower_bound)
    return bounds

def validate_rows_with_bounds(df, bounds):
    error_counts = []
    for index, row in df.iterrows():
        error_count = 0
        upper_bound_index, lower_bound_index = bounds.get(index, (None, None))
        if upper_bound_index is None or lower_bound_index is None:
            error_counts.append(error_count)
            continue
        for col in range(5, df.shape[1] - 4):  # 忽略最后4列的特殊列
            cell = row.iloc[col]
            if cell == '-':
                continue
            upper_cell = df.iloc[upper_bound_index, col] if upper_bound_index else None
            lower_cell = df.iloc[lower_bound_index, col] if lower_bound_index else None
            if upper_cell == '-' or lower_cell == '-':
                continue
            if cell != upper_cell and cell != lower_cell:
                error_count += 1
        error_counts.append(error_count)
    return error_counts

# 预计算每行的上下界索引
bounds = find_bounds_for_all_rows(final_df)

# 使用预计算的索引进行错误计数
error_counts = validate_rows_with_bounds(final_df, bounds)

# 将错误计数添加到DataFrame
final_df['Error_Count2'] = error_counts


def find_bounds_for_all_rows_avoiding_errors(df, bounds, error_threshold=10):
    new_bounds = {}
    for index in bounds:
        # 获取原始的上下界索引
        upper_bound_index, lower_bound_index = bounds[index]

        # 避开Error_Count>10的行，查找新的上界
        while upper_bound_index is not None and df.at[upper_bound_index, 'Error_Count2'] > error_threshold:
            upper_bound_index -= 1  # 往上移动
            if upper_bound_index < 0:
                upper_bound_index = None
                break

        # 避开Error_Count>10的行，查找新的下界
        while lower_bound_index is not None and df.at[lower_bound_index, 'Error_Count2'] > error_threshold:
            lower_bound_index += 1  # 往下移动
            if lower_bound_index >= len(df):
                lower_bound_index = None
                break

        new_bounds[index] = (upper_bound_index, lower_bound_index)
    return new_bounds

def reevaluate_error_counts(df, bounds):
    error_counts = []
    for index, row in df.iterrows():
        upper_bound_index, lower_bound_index = bounds.get(index, (None, None))
        error_count = 0
        if upper_bound_index is None or lower_bound_index is None:
            error_counts.append(error_count)
            continue
        for col in range(5, df.shape[1] - 4):
            cell = row.iloc[col]
            if cell == '-' or df.iloc[upper_bound_index, col] == '-' or df.iloc[lower_bound_index, col] == '-':
                continue
            if cell != df.iloc[upper_bound_index, col] and cell != df.iloc[lower_bound_index, col]:
                error_count += 1
        error_counts.append(error_count)
    return error_counts

# 找出所有Error_Count大于10的行索引
high_error_indices = final_df[final_df['Error_Count2'] > 10].index

# 预计算每行的上下界索引，避开Error_Count>10的行
bounds = find_bounds_for_all_rows(final_df)
new_bounds = find_bounds_for_all_rows_avoiding_errors(final_df, bounds)

# 使用新的上下界索引进行错误计数
new_error_counts = reevaluate_error_counts(final_df, new_bounds)

# 更新DataFrame的Error_Count列
for index, error_count in zip(final_df.index, new_error_counts):
    final_df.at[index, 'Error_Count2'] = error_count

# 处理逻辑完成后，您可以根据需要继续处理df，比如删除Error_Count依然大于10的行


In [6]:
# 保存处理后的数据和失真行
with pd.ExcelWriter("E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx", engine='openpyxl', mode='a') as writer:
    final_df.to_excel(writer, sheet_name='Parent1-6', index=False)

In [8]:
import pandas as pd

# 步骤1: 读取数据集
df_parent1_5 = pd.read_excel('E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx', sheet_name='Parent1-5')
df_parent1_6 = pd.read_excel('E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx', sheet_name='Parent1-6')

# 假设df_parent1_5和df_parent1_6已经是按照相同的顺序排列的，或者它们有共同的键可以合并
# 如果不是，你可能需要通过某个键来合并它们
df_combined = pd.concat([df_parent1_5, df_parent1_6[['Error_Count2']]], axis=1)

# 步骤3: 处理数据
rows_to_delete = []
final_rows = []

for index, row in df_combined.iterrows():
    if row['Error_Count'] > 10 and row['Error_Count2'] > 10:
        # 如果两个错误计数都大于10，则记录该行索引以删除
        rows_to_delete.append(index)
    elif row['Error_Count'] <= 10 or row['Error_Count2'] <= 10:
        # 保留错误计数较小的行
        row_to_keep = row.drop('Error_Count2') if row['Error_Count'] <= row['Error_Count2'] else df_parent1_6.loc[index]
        final_rows.append(row_to_keep)

# 创建最终的DataFrame和删除的行的DataFrame
df_final = pd.DataFrame(final_rows)
df_deleted = df_combined.loc[rows_to_delete]

# 步骤4: 保存结果
with pd.ExcelWriter('E:/My_python_work_xibanya//LG4processed-TEST-erro.xlsx', engine='openpyxl', mode='a') as writer:
    df_final.to_excel(writer, sheet_name='Parent1-7', index=False)
    df_deleted.to_excel(writer, sheet_name='DSC1.1', index=False)
